In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

## Lets get the data, model and setup training code

In [38]:
train_loader = DataLoader(datasets.MNIST("./data", train=True, transform=transforms.ToTensor(), download=True), batch_size=128, shuffle=True)
test_loader = DataLoader(datasets.MNIST("./data", train=False, transform=transforms.ToTensor(), download=True), batch_size=128, shuffle=False)

print(f"Training images {len(train_loader.dataset)}, Test images {len(test_loader.dataset)}")

Training images 60000, Test images 10000


In [39]:
class mnist_model(nn.Module):
  def __init__(self):
    super(mnist_model, self).__init__()
    self.layer1 = nn.Linear(784, 128, bias=True)
    self.layer2 = nn.Linear(128, 128, bias=True)
    self.layer3 = nn.Linear(128, 10, bias=True)
    # self.act = nn.Hardtanh()
    self.act = nn.ReLU()

  def forward(self, x):
    return self.layer3(self.act(self.layer2(self.act(self.layer1(x)))))

  def output(self, x):
    out1 = self.act(self.layer1(x))
    out2 = self.act(self.layer2(out1))
    out3 = self.layer3(out2)
    return out1, out2, out3

In [40]:
model = mnist_model()
print(model)

import os
epochs = 0
lr = 0.1

# GPU run
gpu_list = [0, 1, 2, 3]
gpu_list_str = ','.join(map(str, gpu_list))
os.environ.setdefault("CUDA_VISIBLE_DEVICES", gpu_list_str)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.nn.DataParallel(model)
model.to(device)

USE_MSE = False
optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = nn.MSELoss() if USE_MSE else nn.CrossEntropyLoss()
# lrs = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

mnist_model(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=10, bias=True)
  (act): ReLU()
)


## Training

In [41]:
def get_acc(model, loader):
  correct = 0
  total = 0
  for img, label in loader:
    correct += torch.sum(torch.argmax(model(img.view(-1, 784).to(device)), -1) == label.to(device)).item()
    total += len(img)
  return 100*correct/total

In [42]:
for e in range(epochs):
  print("lr", optimizer.param_groups[0]["lr"])
  for img, label in train_loader:
    out = model(img.view(-1, 784).to(device))
    label_onehot = nn.functional.one_hot(label, num_classes=10).float().to(device)
    optimizer.zero_grad()
    loss = criterion(out, label_onehot)
    loss.backward()
    optimizer.step()
  # lrs.step()
  print(f"Epoch {e}, training accuracy {get_acc(model, train_loader)}, test accuracy {get_acc(model, test_loader)}")

## Extract weights

In [43]:
params = [(name, p.data.cpu().numpy()) for (name, p) in model.named_parameters()]

In [44]:
for (name, p) in params:
  print(f"Layer {name.split('.')[0]}, type {name.split('.')[1]}, shape {p.shape}")

Layer layer1, type weight, shape (128, 784)
Layer layer1, type bias, shape (128,)
Layer layer2, type weight, shape (128, 128)
Layer layer2, type bias, shape (128,)
Layer layer3, type weight, shape (10, 128)
Layer layer3, type bias, shape (10,)


In [45]:
# print(params)

In [46]:
for img, label in train_loader:
  break

In [47]:
# from google.colab import drive
# drive.mount('/content/drive')
# path = "/content/drive/My Drive/Colab Notebooks/Falcon Neural Network/SecureML/"

In [48]:
import os
subdir = 'trained' if epochs != 0 else 'init'
path = f"{os.path.expanduser('~')}/output/{subdir}/SecureML/"
if not os.path.exists(path):
    os.makedirs(path)

print(f'Save parameters to {path}')

np.savetxt(fname=path+"input_0", delimiter=" ", X=img.view(-1, 784).tolist())
np.savetxt(fname=path+"outputlayer1_0", delimiter=" ", X=model.output(img.view(-1, 784).to(device))[0].tolist())
np.savetxt(fname=path+"outputlayer2_0", delimiter=" ", X=model.output(img.view(-1, 784).to(device))[1].tolist())
np.savetxt(fname=path+"outputlayer3_0", delimiter=" ", X=model.output(img.view(-1, 784).to(device))[2].tolist())

np.savetxt(fname=path+"weight1_0", delimiter=" ", X=params[0][1].tolist())
np.savetxt(fname=path+"bias1_0", delimiter=" ",  X=params[1][1].tolist())
np.savetxt(fname=path+"weight2_0", delimiter=" ", X=params[2][1].tolist())
np.savetxt(fname=path+"bias2_0", delimiter=" ", X=params[3][1].tolist())
np.savetxt(fname=path+"weight3_0", delimiter=" ", X=params[4][1].tolist())
np.savetxt(fname=path+"bias3_0", delimiter=" ", X=params[5][1].tolist())

Save parameters to /home/haoqi.whq/output/init/SecureML/
